# Optimization with scalar forecast

In [71]:
import sys
sys.path.append('D:/source/repos')
from utilities.std_imports import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
import supply_chain.scm.inventory_planning as ip
from supply_chain.scm.plan import Plan
import supply_chain.scm.plan_algorithms as pa
from time import time

### Testing

In [72]:
dmd = np.array([0, 130, 160, 120, 260, 130, 120, 135, 105, 110, 195, 100, 110, 105, 110, 110, 115, 118, 105, 125, 122])
plan = Plan(horizon=20, dmd=dmd, on_hand=370)
plan.show()
plan.upd_stk()
plan.show()
plan.show_metrics()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Stock,370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Stock,370,240,80,-40,-300,-430,-550,-685,-790,-900,-1095,-1195,-1305,-1410,-1520,-1630,-1745,-1863,-1968,-2093,-2215


,Metric,Value
0,Total demand,2585.0
1,Average stock,-1002.1
2,N Pl.orders,0.0
3,Holding cost,0.0
4,Transport cost,0.0
5,Total cost,0.0


In [73]:
plan.set_costs(hol_uc=0.8, tr_fix_uc=0.2, tr_var_uc=0.03)
po = np.array([0, 0, 430, 0, 0, 0, 360, 0, 0, 405, 0, 0, 325, 0, 0, 343, 0, 0, 352, 0, 0])
plan.set_po(po)
plan.show()
plan.show_metrics()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,430,0,0,0,360,0,0,405,0,0,325,0,0,343,0,0,352,0,0
Stock,370,240,510,390,130,0,240,105,0,295,100,0,215,110,0,233,118,0,247,122,0


,Metric,Value
0,Total demand,2585.00
1,Average stock,163.10
2,N Pl.orders,6.00
3,Holding cost,137.00
4,Transport cost,3.38
5,Total cost,140.38


In [74]:
end = 12
print('Hol_cost : ', plan.get_hol_cost(end))
print('Tr_cost  : ', plan.get_tr_cost(end))
print('Tot_cost : ', plan.get_tot_cost(end))

Hol_cost :  173.0
Tr_cost  :  3.87
Tot_cost :  176.87


In [75]:
plan.show_ac_costs()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,430,0,0,0,360,0,0,405,0,0,325,0,0,343,0,0,352,0,0
Stock,370,240,510,390,130,0,240,105,0,295,100,0,215,110,0,233,118,0,247,122,0
Ho_cost,0,488,448,402.67,328,262.4,250.67,226.86,198.5,202.67,190.4,173.09,173,166.46,154.57,156.69,152.8,143.81,146.8,144.21,137
Tr_cost,0,0,6.55,4.37,3.27,2.62,4.02,3.44,3.01,4.05,3.65,3.31,3.87,3.57,3.31,3.79,3.56,3.35,3.76,3.56,3.38
Tot_cost,0,488,454.55,407.04,331.27,265.02,254.69,230.3,201.51,206.72,194.05,176.4,176.87,170.03,157.88,160.48,156.36,147.16,150.56,147.77,140.38


### Optimization

In [76]:
plan.set_costs(hol_uc=0.8, tr_fix_uc=0.2, tr_var_uc=0.03)
max_npo = 4
min_repl = 2
max_repl = 4
fst_or_per = pa.get_first_neg(plan)
horizon = plan.horizon
po = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
po[fst_or_per] = 1
po

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [77]:
po = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pbin = pa.brute_force_planning(po=po, max_npo=5, min_repl=2, max_repl=4, horizon=20)
pd.DataFrame(pbin).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0


In [78]:
pidx = pa.pbin2pidx(pbin[3])
pidx

array([ 3,  5,  7, 10, 12, 20], dtype=int64)

In [79]:
po = pa.gen_po(plan, pidx)
plan.set_po(po)
plan.show()
plan.show_metrics()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,380,0,250,0,350,0,0,295,0,898,0,0,0,0,0,0,0,0
Stock,370,240,80,340,80,200,80,295,190,80,180,80,868,763,653,543,428,310,205,80,-42
Ho_cost,0,488,448,402.67,328,262.4,250.67,226.86,198.5,202.67,190.4,173.09,173,166.46,154.57,156.69,152.8,143.81,146.8,144.21,137
Tr_cost,0,0,6.55,4.37,3.27,2.62,4.02,3.44,3.01,4.05,3.65,3.31,3.87,3.57,3.31,3.79,3.56,3.35,3.76,3.56,3.38
Tot_cost,0,488,454.55,407.04,331.27,265.02,254.69,230.3,201.51,206.72,194.05,176.4,176.87,170.03,157.88,160.48,156.36,147.16,150.56,147.77,140.38


,Metric,Value
0,Total demand,2585.00
1,Average stock,286.81
2,N Pl.orders,5.00
3,Holding cost,242.60
4,Transport cost,3.31
5,Total cost,245.91


In [80]:
plan.set_po(po)
plan.show()
plan.show_metrics()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,380,0,250,0,350,0,0,295,0,898,0,0,0,0,0,0,0,0
Stock,370,240,80,340,80,200,80,295,190,80,180,80,868,763,653,543,428,310,205,80,-42
Ho_cost,0,488,448,402.67,328,262.4,250.67,226.86,198.5,202.67,190.4,173.09,173,166.46,154.57,156.69,152.8,143.81,146.8,144.21,137
Tr_cost,0,0,6.55,4.37,3.27,2.62,4.02,3.44,3.01,4.05,3.65,3.31,3.87,3.57,3.31,3.79,3.56,3.35,3.76,3.56,3.38
Tot_cost,0,488,454.55,407.04,331.27,265.02,254.69,230.3,201.51,206.72,194.05,176.4,176.87,170.03,157.88,160.48,156.36,147.16,150.56,147.77,140.38


,Metric,Value
0,Total demand,2585.00
1,Average stock,286.81
2,N Pl.orders,5.00
3,Holding cost,242.60
4,Transport cost,3.31
5,Total cost,245.91


In [81]:
t0 = time()
best_cost, best_po = pa.plan_optimization(plan, pbin)
t1 = time()
print('Time : ', round(t1-t0,2), '\n')
print('Best cost : ', best_cost)
print('Best plan :\n')
plan.set_po(po)
plan.show()  

Time :  0.52 

Best cost :  186.31
Best plan :



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,380,0,250,0,350,0,0,295,0,898,0,0,0,0,0,0,0,0
Stock,370,240,80,340,80,200,80,295,190,80,180,80,868,763,653,543,428,310,205,80,-42
Ho_cost,0,488,448,402.67,328,262.4,250.67,226.86,198.5,202.67,190.4,173.09,173,166.46,154.57,156.69,152.8,143.81,146.8,144.21,137
Tr_cost,0,0,6.55,4.37,3.27,2.62,4.02,3.44,3.01,4.05,3.65,3.31,3.87,3.57,3.31,3.79,3.56,3.35,3.76,3.56,3.38
Tot_cost,0,488,454.55,407.04,331.27,265.02,254.69,230.3,201.51,206.72,194.05,176.4,176.87,170.03,157.88,160.48,156.36,147.16,150.56,147.77,140.38


In [119]:
t0 = time()
costs = np.array([plan.hol_uc, plan.tr_fix_uc, plan.tr_var_uc]).T
best_cost = np.inf
best_po = None
for i in range(len(pbin)-1):
#for i in range(0,1):
    pidx = pa.pbin2pidx(pbin[i])
    po = pa.gen_po(plan, pidx)
    plan.set_po(po)
    acStkPo = np.cumsum(plan.plan.loc['Pl.Or'])
    acDmd = np.cumsum(plan.plan.loc['Dmd'])
    stk = (acStkPo - acDmd) + on_hand
    
    llist = [stk, pbin[i], po]
    P = np.array(llist).T
    cost = np.matmul(P, costs).sum()/(plan.horizon+1)
    #print(cost)
    if(cost < best_cost):
        best_cost = cost
        best_po = po
t1 = time()
print('Time : ', round(t1-t0,2), '\n')
print('Best cost : ', best_cost)
print('Best plan :\n')
plan.set_po(po)
plan.show()  

Time :  0.5 

Best cost :  175.84714285714284
Best plan :



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,630,0,0,0,545,0,0,0,425,0,0,0,343,0,0,230,0,0
Stock,370,240,80,590,330,200,80,490,385,275,80,405,295,190,80,313,198,80,205,80,-42
Ho_cost,0,488,448,402.67,328,262.4,250.67,226.86,198.5,202.67,190.4,173.09,173,166.46,154.57,156.69,152.8,143.81,146.8,144.21,137
Tr_cost,0,0,6.55,4.37,3.27,2.62,4.02,3.44,3.01,4.05,3.65,3.31,3.87,3.57,3.31,3.79,3.56,3.35,3.76,3.56,3.38
Tot_cost,0,488,454.55,407.04,331.27,265.02,254.69,230.3,201.51,206.72,194.05,176.4,176.87,170.03,157.88,160.48,156.36,147.16,150.56,147.77,140.38


In [100]:
pa.pbin2pidx(pbin[0])

array([ 3,  5,  7,  9, 11, 20], dtype=int64)